In [ ]:
%matplotlib inline
import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl

from matplotlib.animation import FuncAnimation
from IPython.display import HTML


In [ ]:
fig_path = pl.Path("../figures")
fig_path.mkdir(exist_ok=True, parents=True)
ani_path = pl.Path("../animation")
ani_path.mkdir(exist_ok=True, parents=True)

# Read the elevation data

In [ ]:
fpath = pl.Path("../data/model_elevations.csv")

In [ ]:
elevation_data = np.genfromtxt(fpath, names=True, delimiter=",")

In [ ]:
elevation_data.dtype.names

# Define the problem dimensions

In [ ]:
km2m = 1000.0
len_x = 10000.0 * km2m
len_z = 20.0 * km2m

In [ ]:
nlay = 20
nrow = 1
ncol = elevation_data["x"].shape[0]
nlay, nrow, ncol

In [ ]:
delx, dely = len_x / ncol, 1.0
delx, dely

In [ ]:
years2days = 365.25
sim_length_years = 1e6
sim_length = sim_length_years * years2days
step_length_years = sim_length_years / 100
nstp = sim_length_years / step_length_years
nper = 1
tdis_data = [(sim_length, nstp, 1.0)]

# Scale the cell center and elevation data using problem dimensions

In [ ]:
x = elevation_data["x"] * len_x

In [ ]:
datum = 0.6 * len_z

In [ ]:
h0 = 0.415 * len_z - datum

In [ ]:
top = elevation_data["top"] * len_z - datum

In [ ]:
bot = elevation_data["bottom"] * len_z - datum

In [ ]:
cryo = elevation_data["cryosphere"] * len_z - datum

In [ ]:
rech = elevation_data["recharge"] * len_z - datum

# Calculate the bottom of each layer and the elevation of each node

In [ ]:
thickness = top - bot
dz = thickness / nlay

In [ ]:
botm = [top - dz * (k + 1) for k in range(nlay)]

In [ ]:
znode = [top - b + dz / 2 for b in botm]

# Plot the model domain

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(
        ncols=1,
        nrows=1,
        layout="constrained",
        figsize=(8, 3),
        )
    ax.set_xlim(0, len_x)
    ax.set_ylim(-13000, 9000)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.fill_between(x, top, y2=cryo, color="green", label="Cryosphere")
    ax.fill_between(x, bot, y2=h0, color="cyan", label="Initial head")
    for b in botm[:-1]:
        ax.plot(x, b, color="black", lw=0.75, label=None)
    ax.axhline(0, color="black", ls="--", label=None)
    ax.plot(-100, 0, color="black", lw=0.75, label="Model layers")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.set_xlabel("x-coordinate, m")
    ax.set_ylabel("Elevation, m")
    flopy.plot.styles.graph_legend(ax=ax, ncol=2, title="", loc="lower left", labelspacing=0.15)
    
    fig.savefig(fig_path / f"conceptual_model.png", dpi=300, transparent=True)
    

# Calculate the depth-dependent permeability, porosity, and specific storage

In [ ]:
10**(-12.65 - 3.2 * np.log10(1))

In [ ]:
logk = np.array([-12.65 - 3.2 * np.log10(z / km2m) for z in znode]).reshape(nlay, nrow, ncol)

In [ ]:
10**logk.mean()

In [ ]:
K = 86400.0 * 1000.0 * 3.7 * 10**logk / 1e-3 # m/d

In [ ]:
logn = np.array([-1.65 - 0.8 * np.log10(z / km2m) for z in znode]).reshape(nlay, nrow, ncol)

In [ ]:
10**logn.mean()

In [ ]:
porosity = 10**logn

In [ ]:
ss = 1000. * 3.7 * (5e-10 + porosity * 4.8e-10)
ss.min(), ss.mean(), ss.max()

# Create the base model

In [ ]:
ws = pl.Path("../run/")
name = "mars"

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws)

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_data)

In [ ]:
ims = flopy.mf6.ModflowIms(
    sim, 
    complexity="simple", 
    linear_acceleration="bicgstab", 
    inner_dvclose=1e-6, 
    outer_dvclose=1e-5,
    outer_maximum=200,
    inner_maximum=100,
    print_option="ALL",
)

## flow model

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, modelname=f"{name}_flow", newtonoptions="under_relaxation")

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf, delr=delx, delc=dely, nlay=nlay, nrow=nrow, ncol=ncol, top=top, botm=botm)

### Plot hydraulic properties using the gwf model

In [ ]:
extent = (0, len_x, -13000, 9000)

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(
        ncols=1,
        nrows=3,
        layout="constrained",
        figsize=(8, 9),
        sharex=True,
        )
    ax = axs[0]
    ax.set_xlim(0, len_x)
    ax.set_ylim(-13000, 9000)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.fill_between(x, top, y2=cryo, color="green", label="Cryosphere")
    ax.fill_between(x, bot, y2=h0, color="cyan", label="Initial head")
    for b in botm[:-1]:
        ax.plot(x, b, color="black", lw=0.75, label=None)
    ax.axhline(0, color="black", ls="--", label=None)
    ax.plot(-100, 0, color="black", lw=0.75, label="Model layers")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.set_ylabel("Elevation, m")
    flopy.plot.styles.graph_legend(ax=ax, ncol=2, title="", loc="lower left", labelspacing=0.15)
    
    ax = axs[1]
    xs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pk = xs.plot_array(logk)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)
    ax.set_ylabel("Elevation, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pk, cax=cax, orientation='horizontal')
    cbar.set_label(r"$\text{log} k\text{, m}^2$")
    flopy.plot.styles.add_text(ax=ax, x=0.75, y=0.9, text=r"$\text{log} k = -12.65 - 3.2 \text{log} z$", bold=False, italic=False)
    
    ax = axs[2]
    xs1 = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pn = xs1.plot_array(logn)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)
    ax.set_ylabel("Elevation, m")
    ax.set_xlabel("x-coordinate, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pn, cax=cax, orientation='horizontal')
    cbar.set_label(r"$\text{log} \theta\text{, unitless}$")
    flopy.plot.styles.add_text(ax=ax, x=0.75, y=0.9, text=r"$\text{log} \theta = -1.65 - 0.8 \text{log} z$", bold=False, italic=False)

    fig.savefig(fig_path / f"model_grid.png", dpi=300, transparent=True)
    

### Build the rest of the flow model

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, k=K, icelltype=1)

In [ ]:
sto = flopy.mf6.ModflowGwfsto(gwf, sy=porosity, ss=ss, transient={0: True}, iconvert=1)

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=h0)

In [ ]:
rch_rate = 2e-10 * years2days
rch_spd = []
for idx, r in enumerate(rech):
    if not np.isnan(r):
        rch_spd.append((0, 0, idx, rch_rate))

In [ ]:
rch = flopy.mf6.ModflowGwfrch(gwf, stress_period_data=rch_spd, pname="RCH-1")

In [ ]:
drn_spd = []
for idx, t in enumerate(top):
    if t < 0.0:
        cond = K[0, 0, idx] * delx * dely / znode[0][idx]
        drn_spd.append((0, 0, idx, float(t), float(cond)))
        

In [ ]:
drn = flopy.mf6.ModflowGwfdrn(gwf, stress_period_data=drn_spd, pname="DRN-1")

In [ ]:
oc = flopy.mf6.ModflowGwfoc(gwf, head_filerecord=f"{name}.hds", saverecord=[("HEAD", "FREQUENCY", "10")])

# Write model datasets and run the model

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation(silent=True)

# Animate results

In [ ]:
ani_ext = ".mp4"
Writer = mpl.animation.writers["ffmpeg"]
writer = Writer(fps=2, metadata=dict(artist="jdhughes"), bitrate=2056)

In [ ]:
output_times = gwf.output.head().get_times()
frames = np.arange(1, len(output_times), dtype=int)

In [ ]:
vmin, vmax = 1e20, -1e20
for totim in output_times:
    head = gwf.output.head().get_data(totim=totim)
    vmin = min(vmin, head[head <= 1e20].min())
    vmax = max(vmax, head[head <= 1e20].max())    
vmin, vmax

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(
        ncols=1,
        nrows=1,
        layout="constrained",
        figsize=(8, 3),
        )
    
    totim = output_times[0]
    toyears = int(totim / years2days)
    title_str = f"{toyears:>,} years"
    fig.suptitle(title_str, fontsize=8)
    
    head = gwf.output.head().get_data(totim=totim)
    
    xs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pk = xs.plot_array(head, head=head, vmin=vmin, vmax=vmax)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)
    ax.set_ylabel("Elevation, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pk, cax=cax, orientation='horizontal')
    cbar.set_label("Head, m")
    flopy.plot.styles.graph_legend(ax=ax, ncol=1, title="", loc="upper right", labelspacing=0.15)

    def func(idx):
        global pk
        totim = output_times[idx]
        toyears = int(totim / years2days)
        title_str = f"{toyears:>,} years"
        fig.suptitle(title_str, fontsize=8)
        
        head = gwf.output.head().get_data(totim=totim)

        pk.remove()
        pk = xs.plot_array(head, head=head)

        return pk

    ani = FuncAnimation(fig, func, frames=frames, blit=False)
    #HTML(ani.to_jshtml())    
    ani.save(ani_path / f"head_results{ani_ext}", writer=writer)